In [13]:
import gradio as gr
import torch
import torch.nn as nn
from torchvision import transforms
from torchvision.models import resnet50
from PIL import Image
import pandas as pd
import cv2
import numpy as np
from ultralytics import YOLO

# --- 1. Tải các mô hình và dữ liệu ---

# CẬP NHẬT ĐƯỜNG DẪN CỦA BẠN TẠI ĐÂY
SIGNNAMES_PATH = r"D:\3\Hoc May\BTL\traffic_sign_project (1)\data\raw_gtsrb\signnames.csv"
CLASSIFIER_MODEL_PATH = r"D:\3\Hoc May\BTL\traffic_sign_project (1)\models\resnet50_best.pth"

# Tải tên các biển báo
try:
    signnames = pd.read_csv(SIGNNAMES_PATH)
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file 'signnames.csv' tại '{SIGNNAMES_PATH}'")
    exit()

# Thiết bị (GPU nếu có)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Sử dụng thiết bị: {device}")

# Tải mô hình YOLOv8 để phát hiện đối tượng (Localization)
# 'yolov8n.pt' là mô hình nhỏ và nhanh, tự động tải về lần đầu chạy
print("Đang tải mô hình phát hiện đối tượng YOLOv8...")
detector_model = YOLO('yolov8n.pt') 
print("Tải YOLOv8 thành công.")

# Tải mô hình ResNet50 đã huấn luyện để phân loại biển báo (Classification)
print("Đang tải mô hình phân loại ResNet50...")
classifier_model = resnet50(weights=None)
classifier_model.fc = nn.Sequential(
    nn.Dropout(0.5),
    nn.Linear(classifier_model.fc.in_features, 43)
)
try:
    classifier_model.load_state_dict(torch.load(CLASSIFIER_MODEL_PATH, map_location=device))
except FileNotFoundError:
    print(f"Lỗi: Không tìm thấy file mô hình phân loại tại '{CLASSIFIER_MODEL_PATH}'")
    exit()
    
classifier_model.to(device)
classifier_model.eval()
print("Tải mô hình phân loại thành công.")


# --- 2. Định nghĩa các phép biến đổi ảnh ---

# Phép biến đổi cho ảnh đầu vào của mô hình phân loại ResNet50
classification_transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

# --- 3. Hàm phát hiện và cắt biển báo bằng YOLOv8 (Nâng cấp cốt lõi) ---

def detect_and_crop_sign_yolo(pil_img):
    """
    Sử dụng YOLOv8 để phát hiện biển báo và trả về ảnh đã được cắt chính xác.
    """
    # YOLOv8 có thể nhận trực tiếp ảnh PIL
    results = detector_model(pil_img, verbose=False) # verbose=False để không in log ra console

    # Lấy các hộp chứa đối tượng (bounding boxes) từ kết quả
    boxes = results[0].boxes
    
    best_box = None
    max_conf = 0

    # Lọc các đối tượng được YOLOv8 nhận diện là "stop sign" (class id 11 trong COCO dataset)
    # hoặc tìm đối tượng có độ tin cậy (confidence) cao nhất nếu không có "stop sign"
    
    # Danh sách các class ID có thể là biển báo trong bộ COCO
    traffic_sign_classes = [11] # 11 là 'stop sign'

    for box in boxes:
        conf = box.conf[0].item()
        cls_id = int(box.cls[0].item())
        
        # Ưu tiên các đối tượng được xác định là biển báo
        if cls_id in traffic_sign_classes:
             if conf > max_conf:
                max_conf = conf
                best_box = box
                
    # Nếu không tìm thấy biển báo cụ thể, hãy lấy đối tượng có độ tin cậy cao nhất
    if best_box is None and len(boxes) > 0:
        # Sắp xếp các box theo độ tin cậy giảm dần và lấy box đầu tiên
        sorted_boxes = sorted(boxes, key=lambda b: b.conf[0].item(), reverse=True)
        best_box = sorted_boxes[0]

    if best_box is not None:
        # Lấy tọa độ xyxy và chuyển sang kiểu integer
        coords = best_box.xyxy[0].cpu().numpy().astype(int)
        # Cắt ảnh gốc bằng tọa độ từ YOLO
        cropped_pil_img = pil_img.crop(tuple(coords))
        return cropped_pil_img

    # Nếu YOLO không phát hiện được gì, trả về None
    return None


# --- 4. Hàm dự đoán chính (sử dụng pipeline mới) ---

def predict(pil_img):
    if pil_img is None:
        return None, "Vui lòng upload ảnh!"

    # Bước 1 (Localization): Dùng YOLO để phát hiện và cắt biển báo
    cropped_pil_img = detect_and_crop_sign_yolo(pil_img)

    if cropped_pil_img is None:
        placeholder_img = Image.new('RGB', (128, 128), color = '#EFEFEF')
        return placeholder_img, "Mô hình YOLOv8 không phát hiện được biển báo nào trong ảnh."

    # Bước 2 (Classification): Dùng ResNet50 để phân loại biển báo đã được cắt
    img_tensor = classification_transform(cropped_pil_img).unsqueeze(0).to(device)

    with torch.no_grad():
        outputs = classifier_model(img_tensor)
        probs = torch.softmax(outputs, dim=1)
        top3_prob, top3_idx = torch.topk(probs, 3)

    # Bước 3: Định dạng kết quả
    results = []
    for prob, idx in zip(top3_prob[0], top3_idx[0]):
        class_id = idx.item()
        class_name = signnames.loc[signnames['ClassId'] == class_id, "SignName"].values[0]
        results.append(f"{class_name} ({prob:.2%})")

    return cropped_pil_img, "\n".join(results)


# --- 5. Xây dựng giao diện Gradio ---
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil", label=" Upload ảnh chứa biển báo giao thông"),
    outputs=[
        gr.Image(type="pil", label=""),
        gr.Textbox(label="")
    ],
    title="🚦 Nhận diện biển báo giao thông (Kiến trúc chuyên nghiệp: YOLOv8 + ResNet50)",
    description="Hệ thống sử dụng YOLOv8 để **định vị** chính xác biển báo trong ảnh, sau đó dùng ResNet50 để **phân loại** chi tiết. Cách tiếp cận này cho độ chính xác cao hơn hẳn.",
    examples=[
        ["stop_sign.jpg"],
        ["speed_limit_sign.jpg"],
        ["yield_sign.jpg"]
    ],
    allow_flagging="never",
    theme=gr.themes.Soft()
)

if __name__ == "__main__":
    demo.launch()

Sử dụng thiết bị: cpu
Đang tải mô hình phát hiện đối tượng YOLOv8...
Tải YOLOv8 thành công.
Đang tải mô hình phân loại ResNet50...
Tải mô hình phân loại thành công.


C:\Users\buih7\AppData\Local\Programs\Python\Python312\Lib\site-packages\gradio\interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


* Running on local URL:  http://127.0.0.1:7867
* To create a public link, set `share=True` in `launch()`.
